想探討91App的會員大部分聚集在哪個地方

In [1]:
import folium
import pandas as pd
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re

In [2]:
df = pd.read_csv('NTU_1317_Member.txt',sep = '\t',encoding = 'utf-8',low_memory=False)
df.head()

,ShopId,MemberId,Recency,Frequency,Monetary,Birthday,GenderTypeDef,LocationCity,LocationState,LocationCountry,...,AnnualIncomeTypeDef,RegisterDateTime,RegisterSourceTypeDef,ValidFlag,MinOrderDate,MaxOrderDate,IsAppInstalled,IsInBlackList,MinAppOpenDateTime,MaxAppOpenDateTime
0,1317,88812F54-9F19-49C0-BC77-86065C356D26,251,3,4011.0,1980-08-08,Female,NaN,台中市,台灣,...,NaN,2015-09-18 19:18:48.050,iOSApp,1,2015-09-18,2017-05-02,有,NaN,2015-09-18 18:45:35.707,2017-07-30 21:30:34.940
1,1317,AB3232A1-03C6-4425-8062-EEECB633FB5A,-1,0,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,2016-03-13 01:11:05.403,Web,1,1900-01-01,NaN,NaN,NaN,NaN,NaN
2,1317,CC63B70D-CD6D-4538-BDDE-4B293F95A654,659,1,529.0,NaN,NaN,NaN,NaN,NaN,...,NaN,2016-03-20 02:20:35.493,Web,1,2016-03-20,2016-03-20,NaN,NaN,NaN,NaN
3,1317,17ED6C4D-0B91-4EB4-9992-117E9440425D,56,3,2438.0,NaN,NaN,NaN,NaN,NaN,...,NaN,2017-06-02 22:16:47.363,Web,1,2017-06-02,2017-10-27,NaN,NaN,NaN,NaN
4,1317,1BA9F6E3-67FD-4AAF-8011-2CBEA98B4828,717,1,550.0,NaN,NaN,NaN,NaN,NaN,...,NaN,2016-01-22 14:46:30.040,Web,1,2016-01-22,2016-01-22,有,NaN,2016-01-22 14:50:32.173,2017-02-21 05:51:05.080


統計每個人住的地方，如果以重複就加一，結果如下，可以看出，資料沒有很統一，例如：會出現Taipei,taipei,台北，但他們都是一樣的地方

In [3]:
location=df['LocationState']
location_dic={}
for place in location:
    if place!="NAN" and place!="nan":
        try:
            place=place.replace(" ", "")
        except:
            pass
        if place not in location_dic:
            location_dic[place]=1
        else:
            location_dic[place]+=1
location_dic

{'': 1,
 ',台北': 1,
 '---': 1,
 '..': 1,
 '...': 1,
 '1234': 1,
 'Chaiyn': 1,
 'Changhua': 3,
 'HSINCHU': 1,
 'HongKong': 4,
 'HsinChu': 1,
 'Hsinchu': 9,
 'Hualien': 3,
 'Huallen': 1,
 'Ilan': 3,
 'KH': 1,
 'KS': 1,
 'Kaoh': 1,
 'Kaohsiung': 39,
 'Kaoshiung': 1,
 'Kaoshon': 1,
 'Keelung': 10,
 'Macau': 1,
 'Melbourne': 1,
 'Miaoli': 1,
 'Mioli': 1,
 'NTc': 1,
 'Nantou': 4,
 'Natou': 1,
 'NewTaipei': 9,
 'PENHU': 1,
 'Pingtong': 1,
 'Pingtung': 1,
 'Singapore': 1,
 'T': 1,
 'TAICHUNG': 1,
 'TAIPEI': 2,
 'TAOYUAN': 2,
 'TEIPEI': 1,
 'TOE': 1,
 'TP': 1,
 'TPE': 5,
 'Ta': 1,
 'Tachung': 1,
 'Taichang': 1,
 'Taichung': 110,
 'Tainai': 1,
 'Tainan': 50,
 'Taioei': 1,
 'Taipai': 3,
 'Taipe': 1,
 'Taipei': 413,
 'Taiwan': 7,
 'Tanain': 1,
 'Taoyuan': 15,
 'Tayouan': 1,
 'Tw': 1,
 'Yilan': 3,
 'Yulin': 1,
 'b': 2,
 'c': 1,
 'changhua': 1,
 'changhwa': 1,
 'chaotic': 1,
 'chayi': 2,
 'chichu': 1,
 'chunhua': 1,
 'hsinchi': 1,
 'hsinchu': 5,
 'hualien': 1,
 'i-lan': 1,
 'k': 2,
 'kH': 1,
 'kaohsi

運用網路上找經緯度的系統，因為要產生的方式是以JS產生，而且不同的地址會有不同的token，所以我運用selenium去尋找各個地址的經緯度，儲存在陣列中

In [9]:
# 用slelnium抓所在地的經緯度，資料過於雜亂==

driver = webdriver.Chrome()
LatLng=[]
persons=[]
for loc in location_dic:
    driver.get("http://itman123.com/show-google-map-latlng.html")
    place_text=loc
    try:
        driver.find_element_by_id("address").send_keys(place_text)
        driver.find_element_by_xpath("//table/tbody/tr[1]/td/p/input[2]").click()
        time.sleep(5)
        try:
            alert = driver.switch_to_alert()
            alert.accept()
            continue
        except:
            soup= BeautifulSoup(driver.page_source, "lxml")
            soup.select("a")
            txt=soup.select("a")[1].get("href")
            try:
                pattern="[0-9]+\.[0-9]+\,[0-9]+\.[0-9]+"
                match=re.search(pattern,txt)
            except:
                pass
            if match[0] not in LatLng:
                LatLng.append(match[0])
                persons.append(int(location_dic[loc]))
            else:
                index=LatLng.index(match[0])
                persons[index]+=int(location_dic[loc])
    except:
        pass
driver.close()

UnexpectedAlertPresentException: Alert Text: None
Message: unexpected alert open: {Alert text : Geocode was not successful for the following reason: ZERO_RESULTS}
  (Session info: chrome=65.0.3325.181)
  (Driver info: chromedriver=2.37.544315 (730aa6a5fdba159ac9f4c1e8cbc59bf1b5ce12b7),platform=Windows NT 10.0.16299 x86_64)


建置一個pandas Dataframe包含經緯度和人數

In [14]:
dic={"place":LatLng,"persons":persons}
loc_df=pd.DataFrame(dic)
loc_df

,persons,place
0,30372,"24.147736,120.673648"
1,4883,"24.813829,120.96748"
2,23632,"25.032969,121.565418"
3,22730,"22.627278,120.301435"
4,4683,"24.838723,121.017725"
5,8513,"24.051796,120.516135"
6,22261,"24.993628,121.30098"
7,4223,"24.560159,120.821427"
8,3478,"24.702107,121.73775"
9,42976,"25.016983,121.462787"


運用folium 在地圖上顯示各地區人數熱擴散圖

In [ ]:
from folium.plugins import HeatMap
import numpy as np
fmap = folium.Map(location=[23.58,120.59], zoom_start=7)
data=[]
for ele in LatLng:
    temp=[]
    eleSplit=ele.split(",")
    temp.append(float(eleSplit[0]))
    temp.append(float(eleSplit[1]))
    temp.append(persons[LatLng.index(ele)])
    data.append(temp)
fmap.add_child(HeatMap(data=data,gradient={.2: 'blue', .5: 'green', 1: 'red'}, min_opacity=0.1,max_val=0.7))

### 後記
    由圖可以看出，紅點集中在北部地區，可以提供給老闆開實體店面的資訊，比較有趣的一點，南投、台東、花蓮的會員比預期的還多，可能的原因是因為那些地方交通不方便，比較常使用網購。然後資料太不統一了，花了很久時間探索資料，建議可以用表單下拉選擇居住地的方式